In [171]:
import gzip
import numpy as np
import csv
import matplotlib.pyplot as plt
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D

In [172]:
# input file using gzip
path = "train_Interactions.csv.gz"
path = "/Users/gakiara/Desktop/258R/assignment1/train_Interactions.csv.gz"
f = gzip.open(path, "rt", encoding="utf8")
reader = csv.reader(f, delimiter=",")

In [173]:
# reading the file to build datasets
datasets = []
first = True
for line in reader:
    if first:
        header = line
        first = False
    else:
        d = dict(zip(header, line))
        # convert strings to integers for some fields
        d["rating"] = int(d["rating"])
        datasets.append(d)

In [174]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.optimize
import random
import csv
import gzip
from collections import defaultdict
import matplotlib.pylab as plt


# input file using gzip
path = "train_Interactions.csv.gz"
path = "/Users/gakiara/Desktop/258R/assignment1/train_Interactions.csv.gz"
f = gzip.open(path, "rt", encoding="utf8")
reader = csv.reader(f, delimiter=",")

# reading the file to build datasets
datasets = []
first = True
for line in reader:
    if first:
        header = line
        first = False
    else:
        d = dict(zip(header, line))
        # convert strings to integers for some fields
        d["rating"] = int(d["rating"])
        datasets.append(d)

size = len(datasets)
allPair = []
for l in datasets:
    user, business, rating = l['userID'], l['bookID'], l['rating']
    allPair.append((user, business, rating))


trainSize = int(0.95 * size)
trainPair = allPair[:trainSize]
validPair = allPair[trainSize:]
uniUser = np.unique([p[0] for p in allPair[:trainSize]])
uniBusi = np.unique([p[1] for p in allPair[:trainSize]])

userBusi_train = defaultdict(list)
busiUser_train = defaultdict(list)
for l in datasets[:trainSize]:
    user, busi, rating = l['userID'], l['bookID'], l['rating']
    userBusi_train[user].append((busi, rating))
    busiUser_train[busi].append((user, rating))


# evaluate performance
def predict(user, busi, theta):
    length = len(theta)
    if length == 3:
        alpha, betaU, betaB = theta
    elif length == 5:
        alpha, betaU, betaB, gammaU, gammaB = theta
    else:
        print
        'theta do not match!'
        return 0
    beta_u = betaU[user] if user in betaU else 0
    beta_b = betaB[busi] if busi in betaB else 0
    gamma_u = gammaU[user] if length == 5 and user in gammaU else np.zeros(1)
    gamma_b = gammaB[busi] if length == 5 and busi in gammaB else np.zeros(1)
    result = alpha + beta_u + beta_b + sum(gamma_u * gamma_b)
    if result < 0.75: result = 0
    if result > 0.75 and result < 1.25:
        if  result > 0.9 and result < 1.1:
            result = 1
        else:
            result = 0
    if result > 1.9 and result < 2.1: result = 2
    if result > 4.9: result = 5
    #    if user in uniUser:
    #        beta_u = betaU[user]
    #        if length == 5:
    #            gamma_u = gammaU[user]
    #    if busi in uniBusi:
    #        beta_b = betaB[busi]
    #        if length == 5:
    #            gamma_b = gammaB[busi]
    return result


def meanSquaredError(dataset, theta):
    size = len(dataset)
    error = 0
    for (user, busi, rating) in dataset:
        prediction = predict(user, busi, theta)
        error = error + (prediction - rating) ** 2 / size
    #        count += 1
    #        if count%5000==0:
    #            print count
    print
    'MSE: ' + str(error)
    return error


# latent factor model (alpha, beta)
def LFM1(lam, max_iter):
    alpha = 0.1
    betaU = {i: 0.0 for i in uniUser}
    betaB = {i: 0.0 for i in uniBusi}
    for it in range(max_iter):
        alpha = 0
        loss = 0
        for (user, busi, rating) in trainPair:
            alpha += (rating - betaU[user] - betaB[busi]) / len(trainPair)
        for (user, busi, rating) in trainPair:
            diff = alpha + betaU[user] + betaB[busi] - rating
            loss += diff ** 2
        squareError = loss
        for (user, busi_list) in userBusi_train.items():
            betaU[user] = 0
            for (busi, rating) in busi_list:
                betaU[user] += (rating - alpha - betaB[busi]) / (lam + \
                                                                 len(busi_list))
            loss += lam * (betaU[user] ** 2)
        for (busi, user_list) in busiUser_train.items():
            betaB[busi] = 0
            for (user, rating) in user_list:
                betaB[busi] += (rating - alpha - betaU[user]) / (lam + \
                                                                 len(user_list))
            loss += lam * (betaB[busi] ** 2)
        if (it + 1) % 5 == 0:
            print('iteration: ' + str(it + 1))
            print(loss/len(trainPair), squareError/len(trainPair))
            meanSquaredError(validPair, [alpha, betaU, betaB])
    return alpha, betaU, betaB

In [175]:
print(len(trainPair))
#alpha, betaU, betaB = LFM1(lam=3.97, max_iter=100)
#MSE_train1 = meanSquaredError(trainPair, [alpha, betaU, betaB])
#MSE_valid1 = meanSquaredError(validPair, [alpha, betaU, betaB])
#print(MSE_train1)
#print(MSE_valid1)

190000


In [176]:
# latent factor model (alpha, beta, gamma)
def LFM2(lam1,lam2, K, learnRate,learnRate2, max_iter):
    errList = []
    alpha = 0
    gammaU = {i: np.random.rand(K) / 10 - 0.05 for i in uniUser}
    gammaB = {i: np.random.rand(K) / 10 - 0.05 for i in uniBusi}
    betaU = {i: 0.2 for i in uniUser}
    betaB = {i: 0.2 for i in uniBusi}
    leng = 10**10
    for it in range(max_iter):
        alpha = 0
        #loss = 0

        gra_gammaU = {user: lam1 * gammaU[user] for user in uniUser}
        gra_gammaB = {busi: lam2 * gammaB[busi] for busi in uniBusi}
        for (user, busi, rating) in trainPair:
            alpha += (rating - betaU[user] - betaB[busi] - np.inner(gammaU[user], gammaB[busi])) / len(trainPair)

        for (user, busi, rating) in trainPair:
            diff = alpha + betaU[user] + betaB[busi] + np.inner(gammaU[user], gammaB[busi]) - rating
            gra_gammaU[user] += gammaB[busi] * diff
            gra_gammaB[busi] += gammaU[user] * diff
            #loss += (diff/leng) ** 2
        #squareError = loss
        for (user, busi_list) in userBusi_train.items():
            gammaU[user] -= learnRate * gra_gammaU[user]
            betaU[user] = 0
            for (busi, rating) in busi_list:
                betaU[user] += (rating - alpha - betaB[busi] - np.inner(gammaU[user], gammaB[busi])) / (
                            lam1 + len(busi_list))
            #loss += lam * ((betaU[user]/leng) ** 2 + sum((gammaU[user]/leng) ** 2))
        for (busi, user_list) in busiUser_train.items():
            gammaB[busi] -= learnRate2 * gra_gammaB[busi]
            betaB[busi] = 0
            for (user, rating) in user_list:
                betaB[busi] += (rating - alpha - betaU[user] - np.inner(gammaU[user], gammaB[busi])) / (
                            lam2 + len(user_list))
            #loss += lam * ((betaB[busi]/leng) ** 2 + sum((gammaB[busi]/leng) ** 2))
        '''
        if squareError*factor < 1 and squareError*factor > 0.9:
            learnRate /= 10
        if squareError*factor < 0.9 and squareError*factor > 0.8:
            learnRate /= 10
        '''
        if (it + 1) % 1 == 0:
            print('iteration: ' + str(it + 1))
            factor = leng**2/len(trainPair)
            #print(loss*factor, squareError*factor)
            err = meanSquaredError(validPair, [alpha, betaU, betaB, gammaU, gammaB])
            if err < 1.3730:
                break
            err2 = meanSquaredError(trainPair, [alpha, betaU, betaB, gammaU, gammaB])
            print(err,err2)
            errList.append(err2)
        if len(errList)>2 and np.abs(errList[-1] - errList[-2]) < 10**-6:
            break


    return alpha, betaU, betaB, gammaU, gammaB

In [177]:
# latent factor model (alpha, beta, gamma)
def LFM_finetune(lam1,lam2, learnRate,learnRate2, max_iter,gU,gB):
    errList = []
    alpha = 0
    gammaU = gU
    gammaB = gB
    betaU = {i: 0.2 for i in uniUser}
    betaB = {i: 0.2 for i in uniBusi}
    leng = 10**10
    for it in range(max_iter):
        alpha = 0
        #loss = 0

        gra_gammaU = {user: lam1 * gammaU[user] for user in uniUser}
        gra_gammaB = {busi: lam2 * gammaB[busi] for busi in uniBusi}
        for (user, busi, rating) in trainPair:
            alpha += (rating - betaU[user] - betaB[busi] - np.inner(gammaU[user], gammaB[busi])) / len(trainPair)

        for (user, busi, rating) in trainPair:
            diff = alpha + betaU[user] + betaB[busi] + np.inner(gammaU[user], gammaB[busi]) - rating
            gra_gammaU[user] += gammaB[busi] * diff
            gra_gammaB[busi] += gammaU[user] * diff
            #loss += (diff/leng) ** 2
        #squareError = loss
        for (user, busi_list) in userBusi_train.items():
            gammaU[user] -= learnRate * gra_gammaU[user]
            betaU[user] = 0
            for (busi, rating) in busi_list:
                betaU[user] += (rating - alpha - betaB[busi] - np.inner(gammaU[user], gammaB[busi])) / (
                            lam1 + len(busi_list))
            #loss += lam * ((betaU[user]/leng) ** 2 + sum((gammaU[user]/leng) ** 2))
        for (busi, user_list) in busiUser_train.items():
            gammaB[busi] -= learnRate2 * gra_gammaB[busi]
            betaB[busi] = 0
            for (user, rating) in user_list:
                betaB[busi] += (rating - alpha - betaU[user] - np.inner(gammaU[user], gammaB[busi])) / (
                            lam2 + len(user_list))
            #loss += lam * ((betaB[busi]/leng) ** 2 + sum((gammaB[busi]/leng) ** 2))
        '''
        if squareError*factor < 1 and squareError*factor > 0.9:
            learnRate /= 10
        if squareError*factor < 0.9 and squareError*factor > 0.8:
            learnRate /= 10
        '''
        if (it + 1) % 1 == 0:
            print('iteration: ' + str(it + 1))
            factor = leng**2/len(trainPair)
            #print(loss*factor, squareError*factor)
            err = meanSquaredError(validPair, [alpha, betaU, betaB, gammaU, gammaB])
            err2 = meanSquaredError(trainPair, [alpha, betaU, betaB, gammaU, gammaB])
            print(err,err2)
            errList.append(err2)
        if len(errList)>2 and np.abs(errList[-1] - errList[-2]) < 10**-6:
            break


    return alpha, betaU, betaB, gammaU, gammaB

In [178]:
print(len(allPair))

200000


In [179]:
Rating_count=defaultdict(list)
for u,b,r in allPair:
    Rating_count[r].append(1)
for r in Rating_count:
    print(r,len(Rating_count[r]))

4 66902
2 12453
3 43432
1 4396
0 10778
5 62039


In [182]:
alpha0,betaU0,betaB0,gammaU0,gammaB0 = LFM2(lam1 = 3.87, lam2 = 14.76, K = 30, learnRate = 0.005,learnRate2 = 0.005, max_iter = 15)
#alpha0,betaU0,betaB0,gammaU0,gammaB0 = LFM2(lam1 = 3.96, lam2 = 14.76, K = 300, learnRate = 0.0005, max_iter = 200)
# LFM2(lam1 = 3.56, lam2 = 14.76, K = 3000, learnRate = 0.0005, max_iter = 100) 1.3733 iter = 10
# LFM2(lam1 = 3.26, lam2 = 14.76, K = 2000, learnRate = 0.0005, max_iter = 100)

iteration: 1
1.3827258703741865 1.1537405636987978
iteration: 2
1.3757159276849154 1.1469655635377698
iteration: 3
1.375225771649253 1.1460429059272632
iteration: 4
1.3745491184079124 1.145270706653409
iteration: 5
1.3742019688760108 1.1445758398960597
iteration: 6
1.3739314955509996 1.1438567424025874
iteration: 7
1.3738687345915905 1.143173234934957
iteration: 8
1.373101333703152 1.1426719499561135
iteration: 9
1.373242899201305 1.1422018098039335
iteration: 10
1.3733159491774865 1.1417153788540841
iteration: 11
1.3733640299998142 1.1412956757421575
iteration: 12
1.3733965963879884 1.1408610999376607
iteration: 13
1.373445347483134 1.1404945205610129
iteration: 14
1.3734904645360375 1.1401211935850077
iteration: 15
1.373529703894917 1.1397495551863555


In [155]:
print(alpha0)
print(gammaU0[0])

3.51947415948313


KeyError: 0

In [ ]:
alpha,betaU,betaB,gammaU,gammaB = LFM_finetune(lam1 = 3.96, lam2 = 14.76, learnRate = 0.00005,learnRate2 = 0.00005, max_iter = 10,gU = gammaU0,gB = gammaB0)
# LFM2(lam1 = 3.56, lam2 = 14.76, K = 3000, learnRate = 0.0005, max_iter = 100) 1.3733 iter = 10
# LFM2(lam1 = 3.26, lam2 = 14.76, K = 2000, learnRate = 0.0005, max_iter = 100)

USE ALL the DATA

In [46]:
'''
trainPair = allPair[:190000]
userBusi_train = defaultdict(list)
busiUser_train = defaultdict(list)
for l in datasets[:trainSize]:
    user, busi, rating = l['userID'], l['bookID'], l['rating']
    userBusi_train[user].append((busi, rating))
    busiUser_train[busi].append((user, rating))
uniUser = np.unique([p[0] for p in allPair[:trainSize]])
uniBusi = np.unique([p[1] for p in allPair[:trainSize]])

'''
trainPair = allPair
userBusi_train = defaultdict(list)
busiUser_train = defaultdict(list)
for l in datasets:
    user, busi, rating = l['userID'], l['bookID'], l['rating']
    userBusi_train[user].append((busi, rating))
    busiUser_train[busi].append((user, rating))
uniUser = np.unique([p[0] for p in allPair])
uniBusi = np.unique([p[1] for p in allPair])


"\ntrainPair = allPair\nuserBusi_train = defaultdict(list)\nbusiUser_train = defaultdict(list)\nfor l in datasets:\n    user, busi, rating = l['userID'], l['bookID'], l['rating']\n    userBusi_train[user].append((busi, rating))\n    busiUser_train[busi].append((user, rating))\nuniUser = np.unique([p[0] for p in allPair])\nuniBusi = np.unique([p[1] for p in allPair])\n"

In [40]:
alpha0,betaU0,betaB0,gammaU0,gammaB0 = LFM2(lam1 = 3.86, lam2 = 13.56, K = 3, learnRate = 0.00005,learnRate2 = 0.0005, max_iter = 10)
#1.47
#alpha0,betaU0,betaB0,gammaU0,gammaB0 = LFM2(lam1 = 3.96, lam2 = 14.76, K = 300, learnRate = 0.0005, max_iter = 200)
# LFM2(lam1 = 3.56, lam2 = 14.76, K = 3000, learnRate = 0.0005, max_iter = 100) 1.3733 iter = 10
# LFM2(lam1 = 3.26, lam2 = 14.76, K = 2000, learnRate = 0.0005, max_iter = 100)

iteration: 1
1.1110213073520616 1.1548764654087507
iteration: 2
1.104555164875171 1.1485405331697904
iteration: 3
1.104299398895047 1.1482355398476325
iteration: 4
1.1041186918334087 1.1480044631011854
iteration: 5
1.1039457393952938 1.1477890759065497
iteration: 6
1.103809227624354 1.147619055347054
iteration: 7
1.1037197915220234 1.1475033889328312
iteration: 8
1.1036736135246767 1.147436534658358
iteration: 9
1.1036607401261422 1.1474072296151365
iteration: 10
1.103670383006681 1.1474037367132064
iteration: 11
1.1036933609773396 1.1474161755023835
iteration: 12
1.1037228144680897 1.1474371558462904
iteration: 13
1.1037540825287366 1.1474616040353074
iteration: 14
1.1037842693469855 1.1474862975520415
iteration: 15
1.103811764911729 1.147509365113758


In [11]:
alpha,betaU,betaB,gammaU,gammaB = alpha0,betaU0,betaB0,gammaU0,gammaB0
# LFM2(lam1 = 3.56, lam2 = 14.76, K = 3000, learnRate = 0.0005, max_iter = 100) 1.3733 iter = 10
# LFM2(lam1 = 3.26, lam2 = 14.76, K = 2000, learnRate = 0.0005, max_iter = 100)

In [44]:
alpha,betaU,betaB,gammaU,gammaB = LFM_finetune(lam1 = 3.87, lam2 = 14.76, learnRate = 0.00005,learnRate2 = 0.0005, max_iter = 10,gU = gammaU0,gB = gammaB0)
# LFM2(lam1 = 3.56, lam2 = 14.76, K = 3000, learnRate = 0.0005, max_iter = 100) 1.3733 iter = 10
# LFM2(lam1 = 3.26, lam2 = 14.76, K = 2000, learnRate = 0.0005, max_iter = 100)

iteration: 1
1.1109270865097334 1.1547747449893662
iteration: 2
1.1044612195814543 1.148439817795654
iteration: 3
1.104206132697047 1.1481356572759822
iteration: 4
1.1040261668947988 1.1479053989302217
iteration: 5
1.1038539684211215 1.147690821828074
iteration: 6
1.1037182130618592 1.147521603413373
iteration: 7
1.103629531106857 1.147406731141399
iteration: 8
1.1035841021940893 1.147340662923793
iteration: 9
1.1035719711495822 1.147312135792794
iteration: 10
1.103582348710757 1.1473094126321457


In [12]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split(',')
    prediction = predict(u,i,(alpha,betaU,betaB,gammaU,gammaB))
    predictions.write(u + ',' + i + ',' + str(prediction) + '\n')
predictions.close()

In [122]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split(',')
    prediction = predict(u,i,(alpha,betaU,betaB,gammaU,gammaB))
    predictions.write(u + ',' + i + ',' + str(prediction) + '\n')
predictions.close()

In [1]:
#finetune the result
predictions = open("predictions_Rating.csv", 'w')
total = 0
for l in open("predictions_Rating2.csv"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b,r = l.strip().split(',')
    r = float(r)
    # because according to the data distribution: most rate are in 3,4,5 ; 1 are the least
    # 0-10778 1-4396 2-12453 3-43432 4-66902 5-62039
    # without: 1.455262755
    # 0.75 : 1.4548354  0.8: 1.4547085
    # 1.5 - 1.451423(70.85%) 1.55-1.450312(59.22%)  1.6 - 1.450312(46.92%) 1.65 - 1.4497803(41%) 1.7- 1.75-1.4484885(37.6%)

    if r < 0.75:  r = 0
    #1.448488
    # 1.5 or 1.55,随机选吧
    if r > 0.75 and r < 1.55:
        if r > 0.9 and r < 1.1:
            #total += (r-1)**2
            r = 1
        else:
            #total += r**2
            r = 0


    #if r > 1.9 and r < 2.1 : r = 2 : 1.4532658
    if r > 1.9 and r < 2.1: r = 2
    if r > 4.9:  r = 5
    predictions.write(u + ',' + b + ',' + str(r) + '\n')
predictions.close()
print(total)

0
